# 📲 YouTube Downloader & Telegram Uploader

This notebook allows you to download a video from YouTube and upload it directly to a Telegram chat. 

**Instructions:**
1. Run the first two code cells to install the necessary libraries and define the functions.
2. In the third cell, enter your YouTube URL, Telegram Bot Token, and Chat ID in the text boxes.
3. Click the "Download and Upload" button to start the process.
4. The output and progress will be displayed below the button.

In [ ]:
# @title 1. Install Dependencies
# This cell installs the required Python libraries.
!pip install yt-dlp python-telegram-bot ipywidgets --quiet

In [ ]:
# @title 2. Define Helper Functions
# This cell defines the Python functions for downloading the video and uploading it to Telegram.

import os
import shutil
import yt_dlp
import telegram
import ipywidgets as widgets
from IPython.display import display, clear_output

def download_video(url, output_widget):
    """
    Downloads a video from the given URL and returns the path to the downloaded file.
    """
    output_path = "downloads"
    
    if os.path.exists(output_path):
        shutil.rmtree(output_path)
    os.makedirs(output_path)

    file_path_container = {}  # Use a dict to store the file path

    def progress_hook(d):
        if d['status'] == 'finished':
            file_path_container['path'] = d.get('filename')
            with output_widget:
                 print(f"✅ Download complete. File saved to: {d.get('filename')}")

    ydl_opts = {
        'outtmpl': os.path.join(output_path, '%(title)s.%(ext)s'),
        'format': 'best[ext=mp4]/best',
        'progress_hooks': [progress_hook],
    }

    with output_widget:
        print(f"⏳ Starting download: {url}")
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        
        return file_path_container.get('path')
    except Exception as e:
        with output_widget:
            print(f"❌ An error occurred during download: {e}")
        return None

def send_file(token, chat_id, file_path, output_widget):
    if not file_path or not os.path.exists(file_path):
        with output_widget:
            print(f"❌ Error: File not found at '{file_path}'")
        return False

    try:
        with output_widget:
            print(f"🔼 Uploading '{os.path.basename(file_path)}' to Telegram...")
        bot = telegram.Bot(token=token)
        with open(file_path, 'rb') as f:
            bot.send_document(chat_id=chat_id, document=f, filename=os.path.basename(file_path))
        with output_widget:
            print(f"✅ File sent successfully to chat '{chat_id}'.")
        return True
    except Exception as e:
        with output_widget:
            print(f"❌ An error occurred during upload: {e}")
        return False

def cleanup(file_path, output_widget):
    with output_widget:
        print("🗑️ Cleaning up local files...")
    try:
        if os.path.exists('downloads'):
            shutil.rmtree('downloads')
        with output_widget:
            print("✅ Cleanup complete.")
    except Exception as e:
        with output_widget:
            print(f"❌ An error occurred during cleanup: {e}")

In [ ]:
# @title 3. Enter Details and Run

# --- Input Fields ---
youtube_url = widgets.Text(description="YouTube URL:", placeholder='https://...', layout=widgets.Layout(width='95%'))
bot_token = widgets.Password(description="Bot Token:", placeholder='Enter your bot token', layout=widgets.Layout(width='95%'))
chat_id = widgets.Text(description="Chat ID:", placeholder='Enter your chat ID', layout=widgets.Layout(width='95%'))

# --- Button and Output ---
button = widgets.Button(description="Download and Upload", button_style='success', icon='upload')
output = widgets.Output()

def run_workflow(b):
    with output:
        clear_output(wait=True)
        url = youtube_url.value
        token = bot_token.value
        cid = chat_id.value
        
        if not all([url, token, cid]):
            print("❌ Please fill in all fields.")
            return
        
        # 1. Download
        file_path = download_video(url, output)
        
        # 2. Upload
        if file_path:
            success = send_file(token, cid, file_path, output)
        
        # 3. Cleanup
        cleanup(file_path, output)
        print("\n✨ Workflow finished!")

button.on_click(run_workflow)

# --- Display UI ---
display(youtube_url, bot_token, chat_id, button, output)